# **EQNeMix**

### EQNmix is a mixed architecture that combines two widely-used neural networks in seismology: ConvNetQuake (Perol et al., 2018) and EQTransformer (Mousavi et al., 2020). Our algorithm employs a Gaussian mixture model for Bayesian Inference using the outputs generated by both neural networks. The ultimate outcome is a probabilistic location pinpointed using just a single seismic station.ks.

##### An integral facet of its versatile design is the algorithm's adaptability, as it is not confined to a single travel-time algorithm. It accommodates a spectrum of options ranging from simpler to more intricate travel-time methods. Furthermore, various sampling techniques such as variational inference, Hamiltonian sampling, among others, can be seamlessly integrated. 
##### This algorithm is applicable not only to individual seismic stations but can also be extended to entire seismic networks.

In [1]:
# Importing libraries
import numpy as np
import pymc3 as pm
import pandas as pd
import json


You can find the C code in this temporary file: /tmp/theano_compilation_error_tdondq6_


Exception: Compilation failed (return status=1): In file included from /Users/cecilia/.theano/compiledir_macOS-12.6-arm64-i386-64bit-i386-3.10.6-64/lazylinker_ext/mod.cpp:1:. /opt/miniforge3/envs/eqt/include/python3.10/Python.h:27:5: error: "Python.h requires that stdio.h define NULL.". #   error "Python.h requires that stdio.h define NULL.".     ^. In file included from /Users/cecilia/.theano/compiledir_macOS-12.6-arm64-i386-64bit-i386-3.10.6-64/lazylinker_ext/mod.cpp:1:. In file included from /opt/miniforge3/envs/eqt/include/python3.10/Python.h:30:. /opt/miniforge3/envs/eqt/bin/../include/c++/v1/string.h:95:102: error: unknown type name 'size_t'. inline _LIBCPP_HIDE_FROM_ABI _LIBCPP_PREFERRED_OVERLOAD const void* memchr(const void* __s, int __c, size_t __n) {.                                                                                                      ^. /opt/miniforge3/envs/eqt/bin/../include/c++/v1/string.h:98:90: error: unknown type name 'size_t'. inline _LIBCPP_HIDE_FROM_

#### **STEP 1:** Obtain events from EQTransformer prediction

In [ ]:
# Read EQT output file
eqt_output = '/Users/jorge/EQTransformer/examples/detectionsCLC/CLC_outputs/X_prediction_results.csv'
df = pd.read_csv(eqt_output)

# Filter events in the dataframe
df_filtered = df[(df['detection_probability'] > 0.95) & 
                 (df['s_probability'] > 0.88) & 
                 (df['p_probability'] > 0.80)]

# Apply UTCDateTime transformation
df_filtered['p_arrival_time'] = pd.to_datetime(df_filtered['p_arrival_time']).apply(UTCDateTime)
df_filtered['s_arrival_time'] = pd.to_datetime(df_filtered['s_arrival_time']).apply(UTCDateTime)

# Calculate the difference between S and P arrival times
df_filtered['t_observed'] = df_filtered['s_arrival_time'] - df_filtered['p_arrival_time']

t_observed = df_filtered['t_observed'].iloc[0]

# Show dataframe filtered
df_filtered

#### **STEP 2:** Upload covariance ellipses information

In [ ]:
# Select reference system: STA or TT
ref = 'TT'

# Choose dimensionality: 2D or 3D
dim = '3D'

# Upload json files
ellipse_data = []
for i in range(6):
    file_path = f'/Users/cecilia/CONVN/data/6_clusters/csv_clusters/{dim}_{ref}/ellipse_parameters_{dim}_{ref}_{i}.json'
    with open(file_path, 'r') as file:
        data = json.load(file)
    datos.append(data)

In [ ]:
# Read covariance matrices
cov_matrices = []
for i in range(6):
    cov_matrices.append(np.array(ellipse_data[i]['Covariance']))

# Show covariance matrices
cov_matrices

In [ ]:
# Read means
mus = []
for i in range(6):
    mus.append(np.array(ellipse_data[i]['Mean']))

# Show means
mus

#### **STEP 3:** Search events in ConvNetQuake results

In [ ]:
# Read CNQ output file
cnq_output = '/Users/cecilia/CONVN/output/july_detections/from_stream/CI.CLC.2019-07-05.csv'
df_cnq = pd.read_csv(cnq_output)

In [ ]:
# Extract P wave arrival times infromation from EQT filtered catalog
p_arrival_time = df_filtered['p_arrival_time'].iloc[1]
p_times = UTCDateTime(p_arrival_time)

# Filter CNQ Dataframe to find where p_times is in between start_time and end_time
find_times = df_cnq[(df_cnq['start_time'] <= p_times) & (df_cnq['end_time'] >= p_times)]


In [ ]:
clusters_prob = search_times['clusters_prob']
clusters_weight = clusters_prob.tolist()[0]
clusters_weight_i = eval(clusters_weight)

w0 = clusters_weight_i[0]
w1 = clusters_weight_i[1]
w2 = clusters_weight_i[2]
w3 = clusters_weight_i[3]
w4 = clusters_weight_i[4]
w5 = clusters_weight_i[5]

weights = [w0, w1, w2, w3, w4, w5] 

#### **STEP 4:** Execute Bayesian Inference

In [ ]:
%%time
# Observed value of S-P is given by EQTransformer [SECONDS]
ts_observed = 35.618300
tp_observed = 34.068300
t_observed = ts_observed - tp_observed
print(f"The t_observed value by EQT is: {t_observed}")

# Define the function S_P_t (Theoretical traveltime function) [SECONDS]
def S_P_t(x, y):
    st_loc = [1, 3]
    p_velocity = 7100   #[METERS/SECOND]
    s_velocity = 2900   #[METERS/SECOND]
    lent = (1 / s_velocity - 1 / p_velocity)
    dis = np.sqrt((x - st_loc[0]) ** 2 + (y - st_loc[1]) ** 2)
    sminp = dis * lent
    return sminp

# Define the Bayesian model
with pm.Model() as model:
    # Define the categories to choose the means
    category = pm.Categorical('category', p=weights)

    # Define the means corresponding to the categories
    mus = [pm.MvNormal(f'mu{i}', mu=mus[i], cov=cov_matrices[i], shape=2) for i in range(len(weights))]

    # Select the averages corresponding to the selected category.
    x = pm.Deterministic('x', pm.math.switch(
        pm.math.eq(category, 0), mus[0][0],
        pm.math.switch(pm.math.eq(category, 1), mus[1][0],
        pm.math.switch(pm.math.eq(category, 2), mus[2][0],
        pm.math.switch(pm.math.eq(category, 3), mus[3][0],
        pm.math.switch(pm.math.eq(category, 4), mus[4][0], mus[5][0]))))))
    
    y = pm.Deterministic('y', pm.math.switch(
        pm.math.eq(category, 0), mus[0][1],
        pm.math.switch(pm.math.eq(category, 1), mus[1][1],
        pm.math.switch(pm.math.eq(category, 2), mus[2][1],
        pm.math.switch(pm.math.eq(category, 3), mus[3][1],
        pm.math.switch(pm.math.eq(category, 4), mus[4][1], mus[5][1]))))))
        
    # Calculate t using the theoretical function
    t = S_P_t(x, y)

    # Likelihood of the observed data
    obs = pm.Normal('obs', mu=t, sigma=0.1, observed=t_observed)

with model:
    trace = pm.sample(300, tune=50, cores=1)

# Results summary
pm.summary(trace)

#pm.traceplot(trace)
#pm.autocorrplot(trace)